In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
import os
from pypapago import Translator

In [2]:
D_eat_info = pd.read_csv("../../Eat/data/DiningCode_df.csv")
D_eat_review = pd.read_csv("../../Eat/data/DiningCode_review_df.csv")
M_eat_info = pd.read_csv("../../Eat/data/MangoPlate_df.csv")
M_eat_review = pd.read_csv("../../Eat/data/MangoPlate_review_df.csv")

In [3]:
#merging data
eat_info = pd.concat([D_eat_info, M_eat_info])
eat_review = pd.concat([D_eat_review, M_eat_review])

In [4]:
eat_info = eat_info[eat_info['name'].notnull()]
eat_review = eat_review[eat_review['user_name'].notnull()]
eat_info.reset_index(drop=True,inplace=True)
eat_review.reset_index(drop=True,inplace=True)

In [5]:
#delete overlapped data
for id in eat_info.index:
    if list(eat_info['name']).count(eat_info.loc[id,'name'])>1:
        eat_info.drop(id,inplace=True)

In [6]:
#label encoding, Place_name -> p_id, User_name -> u_id
le = LabelEncoder()
le.fit(eat_info['name'])
eat_info['p_id'] = le.transform(eat_info['name'])
eat_review['p_id'] = le.transform(eat_review['res_name'])

user_le = LabelEncoder()
user_le.fit(eat_review['user_name'])
eat_review['u_id'] = user_le.transform(eat_review['user_name'])

In [7]:
# delete punctuation marks
def delete_punctuation_marks_review(review):
    review = review.replace('.', '. ');
    review = review.replace('!', '! ');
    review = review.replace('(', '( ');
    review = review.replace(')', ') ');
    review = review.replace('^', '');
    review = review.replace('*', '');
    review = review.replace('-', ' ');
    review = review.replace('\n', '. ');
    review = review.replace('\"', '. ');
    return review

In [8]:
eat_info.head()

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags,p_id
0,놀부만두,서울특별시 동대문구 휘경2동 276-33,NaN,냉모밀,6000.0,"{'월-금,일': '오전 11시 - 오후 8시 30분 '}",NaN,NaN,"점심식사,혼밥,저녁식사,간식,지역주민이 찾는,서민적인,시끌벅적한,가성비좋은,푸짐한,...",721
2,민벅 건대점,서울특별시 광진구 화양동 10-1,NaN,알리오올리오,9900.0,{'매일': '오전 11시 30분 - 오후 10시 (구정 추석 당일 3시 오픈)'},NaN,NaN,"점심식사,저녁식사,데이트,여자끼리,가족외식,모임,기념일,식사모임,아침식사,이국적/이...",1832
3,뉴델리,서울시 동대문구 회기동 3-182,NaN,치킨 탄두리 (한마리),15000.0,{'매일': '오전 11시 - 오후 10시 '},NaN,NaN,"점심식사,저녁식사,데이트,식사모임,이국적/이색적,깔끔한,캐주얼한",741
4,강릉스낵,서울특별시 양천구 목동 406-65,NaN,직송 모듬 회,60000.0,"{'월-금': '오후 5시 - 오전 1시 ', '토/일': '오후 4시 - 오전 1...",NaN,NaN,"술모임,점심식사,저녁식사,간식,데이트,아이동반,가족외식,식사모임,혼밥,서민적인,캐주...",218
7,여기가전집,서울특별시 구로구 구로3동 1124-34,NaN,모둠전,18000.0,{'월-금': '오후 2시 - 오전 2시 '},NaN,NaN,"저녁식사,술모임,식사모임,시끌벅적한,서민적인,가성비좋은,지역주민이 찾는",3276


In [9]:
eat_review.head()

,res_name,user_name,rating,review,p_id,u_id
0,놀부만두,Youngmi Lee,5,지역맛집답게 적당히 허름하고 작지만 손님이 꽉차 있었다. 냉모밀 국물은 깊은달달함~...,721,5351
1,놀부만두,Kunwoo Kim,4,제기동 평양냉면 갔다가 정기휴일 발길돌려휘경동 생방송투데이 맛집 놀부만두로 아버지와...,721,3024
2,놀부만두,erica,3,비빔모밀 온모밀 김치만두 먹어봄유명세에 비해 그닥...그나마 셋 중에는 김치만두가 ...,721,5941
3,놀부만두,발챙이,4,김치만두랑 냉모밀 조합이 최고입니다. 만두피가 엄청 쫄깃해요.,721,13580
4,놀부만두,nelia,5,NaN,721,6953


In [10]:
eat_info['reviews'] = ''

In [11]:
ts = Translator()
eat_review['review_eng'] = ''

#translating review
for id in tqdm(eat_review.index):
    review = eat_review.loc[id,'review']
    
    if type(review)==float:
        continue
        
    review_pre = delete_punctuation_marks_review(review)
    
    try:
        review_ts = ts.translate(review_pre, source='ko', target='en', verbose=False)
    except:
        review_ts = ''
    
    eat_review.loc[id,'review_eng'] = review_ts

In [12]:
for id in eat_info['p_id']:
    review_all = ''
    
    for review in eat_review[eat_review['p_id']==id]['review_eng']:
        review_all+=review +'. '
        
    eat_info.loc[eat_info['p_id']==id,'reviews'] = review_all

In [13]:
eat_info.head()

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags,p_id,reviews
0,놀부만두,서울특별시 동대문구 휘경2동 276-33,NaN,냉모밀,6000.0,"{'월-금,일': '오전 11시 - 오후 8시 30분 '}",NaN,NaN,"점심식사,혼밥,저녁식사,간식,지역주민이 찾는,서민적인,시끌벅적한,가성비좋은,푸짐한,...",721,"As a local restaurant, it was moderately shabb..."
2,민벅 건대점,서울특별시 광진구 화양동 10-1,NaN,알리오올리오,9900.0,{'매일': '오전 11시 30분 - 오후 10시 (구정 추석 당일 3시 오픈)'},NaN,NaN,"점심식사,저녁식사,데이트,여자끼리,가족외식,모임,기념일,식사모임,아침식사,이국적/이...",1832,"If you order 1 menu per person, give me Gorgon..."
3,뉴델리,서울시 동대문구 회기동 3-182,NaN,치킨 탄두리 (한마리),15000.0,{'매일': '오전 11시 - 오후 10시 '},NaN,NaN,"점심식사,저녁식사,데이트,식사모임,이국적/이색적,깔끔한,캐주얼한",741,You can enjoy delicious Indian curry at an old...
4,강릉스낵,서울특별시 양천구 목동 406-65,NaN,직송 모듬 회,60000.0,"{'월-금': '오후 5시 - 오전 1시 ', '토/일': '오후 4시 - 오전 1...",NaN,NaN,"술모임,점심식사,저녁식사,간식,데이트,아이동반,가족외식,식사모임,혼밥,서민적인,캐주...",218,Tteokbokki is spicy and delicious. Fried chick...
7,여기가전집,서울특별시 구로구 구로3동 1124-34,NaN,모둠전,18000.0,{'월-금': '오후 2시 - 오전 2시 '},NaN,NaN,"저녁식사,술모임,식사모임,시끌벅적한,서민적인,가성비좋은,지역주민이 찾는",3276,"The pancakes are delicious~ Potato pancake, pe..."


In [14]:
eat_review.head()

,res_name,user_name,rating,review,p_id,u_id,review_eng
0,놀부만두,Youngmi Lee,5,지역맛집답게 적당히 허름하고 작지만 손님이 꽉차 있었다. 냉모밀 국물은 깊은달달함~...,721,5351,"As a local restaurant, it was moderately shabb..."
1,놀부만두,Kunwoo Kim,4,제기동 평양냉면 갔다가 정기휴일 발길돌려휘경동 생방송투데이 맛집 놀부만두로 아버지와...,721,3024,I went to Pyongyang naengmyeon in Jegi-dong an...
2,놀부만두,erica,3,비빔모밀 온모밀 김치만두 먹어봄유명세에 비해 그닥...그나마 셋 중에는 김치만두가 ...,721,5941,"Among the three, kimchi dumplings were the bes..."
3,놀부만두,발챙이,4,김치만두랑 냉모밀 조합이 최고입니다. 만두피가 엄청 쫄깃해요.,721,13580,The combination of kimchi dumplings and cold b...
4,놀부만두,nelia,5,NaN,721,6953,


In [15]:
eat_info.to_csv('../data/translated_eat_info.csv', index=False)
eat_review.to_csv('../data/translated_eat_review.csv', index=False)